In [3]:
import sqlite3
from sqlite3 import OperationalError

conn = sqlite3.connect('./data/NBAdatabase.db')

In [4]:
import pandas as pd
raw_data = pd.read_csv("raw/2022.txt")
raw_data = raw_data.rename({'Start (ET)': 'start','Visitor/Neutral': 'visitor', 'PTS': 'visitor_pts','Home/Neutral': 'home', 'PTS.1': 'home_pts'}, axis=1)
raw_data = raw_data.drop(['Unnamed: 6', 'Unnamed: 7'], axis=1)
raw_data

,Date,start,visitor,visitor_pts,home,home_pts,Attend.,Notes
0,Tue Oct 19 2021,7:30p,Brooklyn Nets,104,Milwaukee Bucks,127,17341.0,NaN
1,Tue Oct 19 2021,10:00p,Golden State Warriors,121,Los Angeles Lakers,114,18997.0,NaN
2,Wed Oct 20 2021,7:00p,Indiana Pacers,122,Charlotte Hornets,123,15521.0,NaN
3,Wed Oct 20 2021,7:00p,Chicago Bulls,94,Detroit Pistons,88,20088.0,NaN
4,Wed Oct 20 2021,7:30p,Boston Celtics,134,New York Knicks,138,19812.0,NaN
...,...,...,...,...,...,...,...,...
1145,Thu Mar 31 2022,7:00p,Philadelphia 76ers,94,Detroit Pistons,102,20023.0,NaN
1146,Thu Mar 31 2022,7:30p,Cleveland Cavaliers,107,Atlanta Hawks,131,17491.0,NaN
1147,Thu Mar 31 2022,7:30p,Milwaukee Bucks,120,Brooklyn Nets,119,17917.0,NaN
1148,Thu Mar 31 2022,8:00p,Los Angeles Clippers,130,Chicago Bulls,135,21519.0,NaN


In [5]:
raw_data["Date"]= pd.to_datetime(raw_data["Date"])
raw_data['start'] = pd.to_datetime(raw_data['start'])
raw_data['start'] = [time.time() for time in raw_data['start']]
raw_data

,Date,start,visitor,visitor_pts,home,home_pts,Attend.,Notes
0,2021-10-19,19:30:00,Brooklyn Nets,104,Milwaukee Bucks,127,17341.0,NaN
1,2021-10-19,22:00:00,Golden State Warriors,121,Los Angeles Lakers,114,18997.0,NaN
2,2021-10-20,19:00:00,Indiana Pacers,122,Charlotte Hornets,123,15521.0,NaN
3,2021-10-20,19:00:00,Chicago Bulls,94,Detroit Pistons,88,20088.0,NaN
4,2021-10-20,19:30:00,Boston Celtics,134,New York Knicks,138,19812.0,NaN
...,...,...,...,...,...,...,...,...
1145,2022-03-31,19:00:00,Philadelphia 76ers,94,Detroit Pistons,102,20023.0,NaN
1146,2022-03-31,19:30:00,Cleveland Cavaliers,107,Atlanta Hawks,131,17491.0,NaN
1147,2022-03-31,19:30:00,Milwaukee Bucks,120,Brooklyn Nets,119,17917.0,NaN
1148,2022-03-31,20:00:00,Los Angeles Clippers,130,Chicago Bulls,135,21519.0,NaN


In [6]:
from sklearn.preprocessing import LabelEncoder

raw_data['datetime'] = pd.to_datetime(raw_data["Date"].astype(str) + " " + raw_data["start"].astype(str))



In [7]:
total_lst = []
for row in raw_data.index:
    total_lst.append((raw_data['datetime'][row], 
                           raw_data['home'][row], 
                           raw_data['home_pts'][row], 
                           raw_data['visitor'][row], 
                           raw_data['visitor_pts'][row] ))
new_lst = []
name_lst = []
for row in total_lst:
    if row[0].day <=20 and row[0].month == 10:
        new_lst.append(row)
    if row[1] not in name_lst:
        name_lst.append(row[1])
    if row[3] not in name_lst:
        name_lst.append(row[3])
total_lst = new_lst

le =  LabelEncoder()
encoded_labels = le.fit_transform(name_lst)
labels_dict = {}
for i in range(len(name_lst)):
    labels_dict[name_lst[i]] = encoded_labels[i]
    
print(labels_dict)


{'Milwaukee Bucks': 16, 'Brooklyn Nets': 2, 'Los Angeles Lakers': 13, 'Golden State Warriors': 9, 'Charlotte Hornets': 3, 'Indiana Pacers': 11, 'Detroit Pistons': 8, 'Chicago Bulls': 4, 'New York Knicks': 19, 'Boston Celtics': 1, 'Toronto Raptors': 27, 'Washington Wizards': 29, 'Memphis Grizzlies': 14, 'Cleveland Cavaliers': 5, 'Minnesota Timberwolves': 17, 'Houston Rockets': 10, 'New Orleans Pelicans': 18, 'Philadelphia 76ers': 22, 'San Antonio Spurs': 26, 'Orlando Magic': 21, 'Utah Jazz': 28, 'Oklahoma City Thunder': 20, 'Portland Trail Blazers': 24, 'Sacramento Kings': 25, 'Phoenix Suns': 23, 'Denver Nuggets': 7, 'Atlanta Hawks': 0, 'Dallas Mavericks': 6, 'Miami Heat': 15, 'Los Angeles Clippers': 12}


 Create Table "Team"

In [9]:
conn.execute('''DROP TABLE team;''')
conn.execute('''CREATE TABLE team(
                TeamId INTEGER NOT NULL,
                Name VARCHAR,
                PRIMARY KEY (TeamId)
                )''')

for ele in labels_dict:
    conn.execute( ''' INSERT INTO team(TeamId, Name) VALUES(?,?) ''', (int(labels_dict[ele]), ele))

In [10]:
results = list(conn.execute( ''' SELECT * FROM team'''))
print(results)

[(0, 'Atlanta Hawks'), (1, 'Boston Celtics'), (2, 'Brooklyn Nets'), (3, 'Charlotte Hornets'), (4, 'Chicago Bulls'), (5, 'Cleveland Cavaliers'), (6, 'Dallas Mavericks'), (7, 'Denver Nuggets'), (8, 'Detroit Pistons'), (9, 'Golden State Warriors'), (10, 'Houston Rockets'), (11, 'Indiana Pacers'), (12, 'Los Angeles Clippers'), (13, 'Los Angeles Lakers'), (14, 'Memphis Grizzlies'), (15, 'Miami Heat'), (16, 'Milwaukee Bucks'), (17, 'Minnesota Timberwolves'), (18, 'New Orleans Pelicans'), (19, 'New York Knicks'), (20, 'Oklahoma City Thunder'), (21, 'Orlando Magic'), (22, 'Philadelphia 76ers'), (23, 'Phoenix Suns'), (24, 'Portland Trail Blazers'), (25, 'Sacramento Kings'), (26, 'San Antonio Spurs'), (27, 'Toronto Raptors'), (28, 'Utah Jazz'), (29, 'Washington Wizards')]


In [11]:
conn.execute('''DROP TABLE schedule;''')
conn.execute('''CREATE TABLE schedule(
                GameId INTEGER NOT NULL,
                date TEXT NOT NULL,
                homeId INTEGER,
                home_pts INTEGER,
                visitorId INTEGER,
                visitor_pts INTEGER,
                FOREIGN KEY ([homeId]) REFERENCES [team] ([TeamId]) ON DELETE NO ACTION ON UPDATE NO ACTION,
                FOREIGN KEY ([visitorId]) REFERENCES [team] ([TeamId]) ON DELETE NO ACTION ON UPDATE NO ACTION,
                PRIMARY KEY (GameId)
                )''')

for num, ele in enumerate(total_lst):
    dt = ele[0].strftime('%Y-%m-%d %H:%M:%S')
    conn.execute( ''' INSERT INTO schedule(GameId, date, homeId, home_pts, visitorId, visitor_pts) VALUES(?,?,?,?,?,?) ''', (num, dt, int(labels_dict[ele[1]]), int(ele[2]), int(labels_dict[ele[3]]), int(ele[4])))

In [12]:
results = list(conn.execute( ''' SELECT * FROM schedule'''))
print(results)

[(0, '2021-10-19 19:30:00', 16, 127, 2, 104), (1, '2021-10-19 22:00:00', 13, 114, 9, 121), (2, '2021-10-20 19:00:00', 3, 123, 11, 122), (3, '2021-10-20 19:00:00', 8, 88, 4, 94), (4, '2021-10-20 19:30:00', 19, 138, 1, 134), (5, '2021-10-20 19:30:00', 27, 83, 29, 98), (6, '2021-10-20 20:00:00', 14, 132, 5, 121), (7, '2021-10-20 20:00:00', 17, 124, 10, 106), (8, '2021-10-20 20:00:00', 18, 97, 22, 117), (9, '2021-10-20 20:30:00', 26, 123, 21, 97), (10, '2021-10-20 21:00:00', 28, 107, 20, 86), (11, '2021-10-20 22:00:00', 24, 121, 25, 124), (12, '2021-10-20 22:00:00', 23, 98, 7, 110)]


In [13]:
import os

root = r"raw/new/"
file_list = [os.path.join(path, name) for path, subdirs, files in os.walk(root) for name in files]
print(file_list)

['raw/new/19LL_adv.csv', 'raw/new/20UJ_basic.csv', 'raw/new/20CB_adv.csv', 'raw/new/20MG_adv.csv', 'raw/new/20BC_basic.csv', 'raw/new/20SS_adv.csv', 'raw/new/20BC_adv.csv', 'raw/new/20CC_adv.csv', 'raw/new/20WW_adv.csv', 'raw/new/20OT_adv.csv', 'raw/new/20PB_adv.csv', 'raw/new/20CH_adv.csv', 'raw/new/20P7_basic.csv', 'raw/new/20NK_basic.csv', 'raw/new/20NK_adv.csv', 'raw/new/20PS_basic.csv', 'raw/new/20TR_adv.csv', 'raw/new/19MB_adv.csv', 'raw/new/20SK_basic.csv', 'raw/new/19GW_basic.csv', 'raw/new/20MT_adv.csv', 'raw/new/20SS_basic.csv', 'raw/new/20DN_adv.csv', 'raw/new/19LL_basic.csv', 'raw/new/20IP_adv.csv', 'raw/new/19GW_adv.csv', 'raw/new/20PS_adv.csv', 'raw/new/20WW_basic.csv', 'raw/new/20UJ_adv.csv', 'raw/new/20P7_adv.csv', 'raw/new/20TR_basic.csv', 'raw/new/19BN_adv.csv', 'raw/new/20CH_basic.csv', 'raw/new/20OT_basic.csv', 'raw/new/20NP_basic.csv', 'raw/new/20IP_basic.csv', 'raw/new/20CB_basic.csv', 'raw/new/20OM_adv.csv', 'raw/new/20MG_basic.csv', 'raw/new/20PB_basic.csv', 'ra

In [14]:
import pandas as pd
player_lst = []
print(file_list)



for file in file_list:
    df = pd.read_csv(file)
    for index, row in df.iterrows():
        if row['Unnamed: 0'] != 'Starters'and row['Unnamed: 0'] not in player_lst:
            player_lst.append(row['Unnamed: 0'])

print(len(player_lst))
player_dict = {}
for idx, name in enumerate(player_lst):
    player_dict[name] = idx
    
player_dict

['raw/new/19LL_adv.csv', 'raw/new/20UJ_basic.csv', 'raw/new/20CB_adv.csv', 'raw/new/20MG_adv.csv', 'raw/new/20BC_basic.csv', 'raw/new/20SS_adv.csv', 'raw/new/20BC_adv.csv', 'raw/new/20CC_adv.csv', 'raw/new/20WW_adv.csv', 'raw/new/20OT_adv.csv', 'raw/new/20PB_adv.csv', 'raw/new/20CH_adv.csv', 'raw/new/20P7_basic.csv', 'raw/new/20NK_basic.csv', 'raw/new/20NK_adv.csv', 'raw/new/20PS_basic.csv', 'raw/new/20TR_adv.csv', 'raw/new/19MB_adv.csv', 'raw/new/20SK_basic.csv', 'raw/new/19GW_basic.csv', 'raw/new/20MT_adv.csv', 'raw/new/20SS_basic.csv', 'raw/new/20DN_adv.csv', 'raw/new/19LL_basic.csv', 'raw/new/20IP_adv.csv', 'raw/new/19GW_adv.csv', 'raw/new/20PS_adv.csv', 'raw/new/20WW_basic.csv', 'raw/new/20UJ_adv.csv', 'raw/new/20P7_adv.csv', 'raw/new/20TR_basic.csv', 'raw/new/19BN_adv.csv', 'raw/new/20CH_basic.csv', 'raw/new/20OT_basic.csv', 'raw/new/20NP_basic.csv', 'raw/new/20IP_basic.csv', 'raw/new/20CB_basic.csv', 'raw/new/20OM_adv.csv', 'raw/new/20MG_basic.csv', 'raw/new/20PB_basic.csv', 'ra

{'Anthony Davis': 0,
 'LeBron James': 1,
 'Russell Westbrook': 2,
 'Kent Bazemore': 3,
 'DeAndre Jordan': 4,
 'Carmelo Anthony': 5,
 'Rajon Rondo': 6,
 'Malik Monk': 7,
 'Dwight Howard': 8,
 'Avery Bradley': 9,
 'Austin Reaves': 10,
 'Sekou Doumbouya': 11,
 'Rudy Gobert': 12,
 'Donovan Mitchell': 13,
 'Bojan BogdanoviÄ‡': 14,
 "Royce O'Neale": 15,
 'Mike Conley': 16,
 'Jordan Clarkson': 17,
 'Joe Ingles': 18,
 'Hassan Whiteside': 19,
 'Eric Paschall': 20,
 'Jared Butler': 21,
 'Elijah Hughes': 22,
 'Malik Fitts': 23,
 'Miye Oni': 24,
 'Udoka Azubuike': 25,
 'DeMar DeRozan': 26,
 'Nikola VuÄ\x8deviÄ‡': 27,
 'Zach LaVine': 28,
 'Lonzo Ball': 29,
 'Patrick Williams': 30,
 'Alex Caruso': 31,
 'Javonte Green': 32,
 'Alize Johnson': 33,
 'Ayo Dosunmu': 34,
 'Troy Brown Jr.': 35,
 'Marko Simonovic': 36,
 'Tony Bradley': 37,
 'Matt Thomas': 38,
 'Derrick Jones Jr.': 39,
 'Ja Morant': 40,
 'Steven Adams': 41,
 'Jaren Jackson Jr.': 42,
 "De'Anthony Melton": 43,
 'Desmond Bane': 44,
 'Kyle Anders

In [44]:
# conn.execute('''DROP TABLE player;''')
conn.execute('''CREATE TABLE player(
                PlayerId INTEGER NOT NULL,
                Name VARCHAR,
                PRIMARY KEY (PlayerId)
                )''')

for ele in player_dict:
    conn.execute( ''' INSERT INTO player(PlayerId, Name) VALUES(?,?) ''', (int(player_dict[ele]), ele))

In [45]:
results = list(conn.execute( ''' SELECT * FROM player'''))
print(results)

[(0, 'Anthony Davis'), (1, 'LeBron James'), (2, 'Russell Westbrook'), (3, 'Kent Bazemore'), (4, 'DeAndre Jordan'), (5, 'Carmelo Anthony'), (6, 'Rajon Rondo'), (7, 'Malik Monk'), (8, 'Dwight Howard'), (9, 'Avery Bradley'), (10, 'Austin Reaves'), (11, 'Sekou Doumbouya'), (12, 'Rudy Gobert'), (13, 'Donovan Mitchell'), (14, 'Bojan BogdanoviÄ‡'), (15, "Royce O'Neale"), (16, 'Mike Conley'), (17, 'Jordan Clarkson'), (18, 'Joe Ingles'), (19, 'Hassan Whiteside'), (20, 'Eric Paschall'), (21, 'Jared Butler'), (22, 'Elijah Hughes'), (23, 'Malik Fitts'), (24, 'Miye Oni'), (25, 'Udoka Azubuike'), (26, 'DeMar DeRozan'), (27, 'Nikola VuÄ\x8deviÄ‡'), (28, 'Zach LaVine'), (29, 'Lonzo Ball'), (30, 'Patrick Williams'), (31, 'Alex Caruso'), (32, 'Javonte Green'), (33, 'Alize Johnson'), (34, 'Ayo Dosunmu'), (35, 'Troy Brown Jr.'), (36, 'Marko Simonovic'), (37, 'Tony Bradley'), (38, 'Matt Thomas'), (39, 'Derrick Jones Jr.'), (40, 'Ja Morant'), (41, 'Steven Adams'), (42, 'Jaren Jackson Jr.'), (43, "De'Anthony

In [46]:
conn.commit()
conn.close()

Take Action

In [49]:
import sqlite3
from sqlite3 import OperationalError

conn = sqlite3.connect('./data/NBAdatabase.db')

In [78]:

from datetime import datetime

def track_game_id(game_lst, team_id, filename):
    for ele in game_lst:
        dt = datetime.strptime(ele[1], "%Y-%m-%d %H:%M:%S")
        if str(dt.day) == filename[8:10] and (ele[2] == team_id or ele[4] == team_id):
            return ele[0]

conn.execute('''DROP TABLE advance_data;''')
conn.execute('''CREATE TABLE advance_data(
                GameId INTEGER NOT NULL,
                PlayerId INTEGER NOT NULL,
                TeamId INTEGER
                TrueShooting DECIMAL,
                EffectiveFieldGoal DECIMAL,
                ThreePointAttemptRate DECIMAL,
                FreeThrowRatio DECIMAL,
                DefensiveReboundPercentage DECIMAL,
                OffensiveReboundPercentage DECIMAL,
                TeamAssistsPercentage DECIMAL,
                StealPercentage DECIMAL,
                BlockPercentage DECIMAL,
                PRIMARY KEY (GameId, PlayerId)
                )''')


list1 = [(0, '2021-10-19 19:30:00', 16, 127, 2, 104), (1, '2021-10-19 22:00:00', 13, 114, 9, 121), (2, '2021-10-20 19:00:00', 3, 123, 11, 122), (3, '2021-10-20 19:00:00', 8, 88, 4, 94), (4, '2021-10-20 19:30:00', 19, 138, 1, 134), (5, '2021-10-20 19:30:00', 27, 83, 29, 98), (6, '2021-10-20 20:00:00', 14, 132, 5, 121), (7, '2021-10-20 20:00:00', 17, 124, 10, 106), (8, '2021-10-20 20:00:00', 18, 97, 22, 117), (9, '2021-10-20 20:30:00', 26, 123, 21, 97), (10, '2021-10-20 21:00:00', 28, 107, 20, 86), (11, '2021-10-20 22:00:00', 24, 121, 25, 124), (12, '2021-10-20 22:00:00', 23, 98, 7, 110)] 

for file in file_list: 
    if file[13:16] == 'adv':
        team_name = ''
        for ele in list(labels_dict.keys()):
            name = ele.split(' ')
            if file[10] == name[0][0] and file[11] == name[-1][0]:
                teamid = labels_dict[ele]
        df = pd.read_csv(file)
        for index, row in df.iterrows():
            if row[0] != 'Starters':
                row = list(row)
                print(row)
                game_id = track_game_id(list1, teamid, file)
                print(game_id)
                print((game_id, int(player_dict[row[0]]), int(teamid), float(row[2]), float(row[3]), float(row[4]), float(row[5]), float(row[6]), float(row[7]), float(row[8]), float(row[9]), float(row[10])))
#                 conn.execute( ''' INSERT INTO advance_data(GameId, PlayerId, TeamId, TrueShooting, EffectiveFieldGoal, ThreePointAttemptRate, FreeThrowRatio, DefensiveReboundPercentage, OffensiveReboundPercentage, TeamAssistsPercentage, StealPercentage, BlockPercentage) VALUES(?,?,?,?,?,?,?,?,?,?,?,?) ''', (game_id, int(player_dict[ele[0]]), int(teamid), float(row[2]), float(row[3]), float(row[4]), float(row[5]), float(row[6]), float(row[7]), float(row[8]), float(row[9]), float(row[10])))
                             
                             

['Anthony Davis', '38:55', '0.567', '0.596', '0.192', '0.269', '5.4', '22.7', '14.3', '9.3', '1.1', '4.6', '0', '29.8', '116', '105', '8.2']
1
(1, 0, 13, 0.567, 0.596, 0.192, 0.269, 5.4, 22.7, 14.3, 9.3, 1.1)
['LeBron James', '36:44', '0.663', '0.674', '0.478', '0.261', '2.8', '26.7', '15.1', '23.3', '1.2', '2.4', '13.5', '32.2', '117', '105', '10.3']
1
(1, 1, 13, 0.663, 0.674, 0.478, 0.261, 2.8, 26.7, 15.1, 23.3, 1.2)
['Russell Westbrook', '35:08', '0.308', '0.308', '0.308', '0', '3', '11.2', '7.2', '13.8', '0', '0', '23.5', '19.3', '63', '113', '-12.6']
1
(1, 2, 13, 0.308, 0.308, 0.308, 0.0, 3.0, 11.2, 7.2, 13.8, 0.0)
['Kent Bazemore', '30:37', '0.444', '0.444', '0.889', '0', '0', '6.4', '3.3', '0', '0', '2.9', '10', '13', '77', '112', '-5.8']
1
(1, 3, 13, 0.444, 0.444, 0.889, 0.0, 0.0, 6.4, 3.3, 0.0, 0.0)
['DeAndre Jordan', '12:55', '1', '1', '0', '0', '0', '15.2', '7.8', '9', '3.3', '0', '0', '3.1', '209', '104', '2']
1
(1, 4, 13, 1.0, 1.0, 0.0, 0.0, 0.0, 15.2, 7.8, 9.0, 3.3)
['Car

In [19]:
conn.commit()
conn.close()